In [37]:
import subprocess
import SimpleITK as sitk
import cv2
import numpy as np
import h5py
import os

In [33]:
#Paths

mhdInputFile = 'D:\\dataset_seg\\xVertSeg.v1\\Data1\\images\\image004.mhd'
pngDir = 'D:\\dataset_seg\\data\\imageSlices_004\\'

ilastik_location = 'C:\\Program Files\\ilastik-1.3.2post1'
ilastik_project = 'D:\\dataset_seg\\Ilastik\\proj1_b_2.ilp'

indir = 'D:\\dataset_seg\\data\\imageSlices_004\\' 
outdir = 'D:\\dataset_seg\\test\\'

infiles = "*.png"
outputFileName = "results004"

In [36]:
#creating pngDir
if not (os.path.exists(pngDir) and os.path.isdir(pngDir)):
    print("Creating directory: " + pngDir)
    try:
        os.mkdir(pngDir)
    except Exception as e:
        print("Error creating directory: " + pngDir + "\n" + e)   

Creating directory: D:\dataset_seg\data\imageSlices_006\


In [26]:
#.mhd to .pngs

def convertToGrayScaleRange(image, m):
    new_img = np.array(image)
    new_img[:, :] = (image/m)*255
    return new_img


img = sitk.ReadImage(mhdInputFile)
img_spacing = img.GetSpacing()

no = sitk.GetArrayViewFromImage(img).shape[0]
shape = 0
m = 0
for j in range(no):
    view = sitk.GetArrayViewFromImage(img)[j, :, :]
    if max(np.unique(view)) > m:
        m = max(np.unique(view))   

for k in range(no):
    view = sitk.GetArrayViewFromImage(img)[k, :, :]
    grayView = convertToGrayScaleRange(view, m)
    ind = str(k+1)
    if len(ind) < 3:
        ind = "0"*(3 - len(ind)) + ind
    grayView = cv2.resize(grayView, (1024, 1024)) 
    cv2.imwrite(pngDir + "slice" + ind + ".png", grayView)

In [18]:
#using the ilastik project for predictions 

os.chdir(ilastik_location)
command = 'ilastik.exe --headless '
command += '--project="' + ilastik_project + '" '
command += '--export_source="probabilities" ' 
command += '--export_dtype="float32" ' 
command += '--output_filename_format="' + outdir + outputFileName + '" ' 
command += '--raw_data="' + indir + infiles + '"'
print("\n\n%s" % command)
#subprocess.call(command, shell=True)



ilastik.exe --headless --project="D:\dataset_seg\Ilastik\proj1_b_2.ilp" --export_source="probabilities" --export_dtype="float32" --output_filename_format="D:\dataset_seg\test\results004" --raw_data="D:\dataset_seg\data\imageSlices_004\*.png"


In [29]:
#hdf5 to .mhd

#Convert prob map to B&W
def conv(x):    
    a = x[:, :, 0]
    b = x[:, :, 1]
    a[a > 0.5] = True
    a[a <= 0.5] = False
    return a

hdf = h5py.File(outdir + outputFileName + '.h5', 'r')
data = hdf['exported_data']

newData = []
for i in range(data.shape[0]):
    newData.append(conv(data[i]))

newData = np.array(newData)
newImg = sitk.GetImageFromArray(newData)
newImg.SetSpacing(img_spacing)

sitk.WriteImage(newImg, outdir + outputFileName + ".mhd")

In [28]:
np.unique(data[i])


array([0.        , 0.01      , 0.02      , 0.03      , 0.04      ,
       0.05      , 0.06      , 0.07      , 0.08      , 0.09      ,
       0.1       , 0.11      , 0.12      , 0.13      , 0.14      ,
       0.15      , 0.16      , 0.17      , 0.18      , 0.19      ,
       0.2       , 0.21      , 0.22      , 0.23      , 0.24      ,
       0.25      , 0.26      , 0.27      , 0.28      , 0.29      ,
       0.3       , 0.31      , 0.32      , 0.33      , 0.34      ,
       0.35      , 0.36      , 0.37      , 0.38      , 0.39      ,
       0.4       , 0.41      , 0.42      , 0.43      , 0.44      ,
       0.45      , 0.46      , 0.47      , 0.48      , 0.49      ,
       0.5       , 0.51      , 0.52      , 0.53      , 0.53000003,
       0.54      , 0.55      , 0.56      , 0.56000006, 0.57      ,
       0.57000005, 0.58      , 0.58000004, 0.59      , 0.59000003,
       0.6       , 0.61      , 0.62      , 0.62000006, 0.63      ,
       0.63000005, 0.64      , 0.65      , 0.66      , 0.67   